# help connected - data schema generation/exploration
## Scope
For each step of the user flow - 
- propose data points collected.
- draft a database schema to hold them.

## User Flow
![alt text](static/userflow.jpg)

## Data capture opportunities
- Registration of new users
- Geocoding of suburb or postcode
- Confirmation of volunteers

## Setup - API and database connection

In [1]:
import requests # library for API requests
import json
import pandas as pd # flat file handling and manipulation
import psycopg2 # engine for directly writing to database
import sqlalchemy # engine for engaging with database through pandas
from faker import Faker # for generating data
import configparser # parses a Java config file

In [2]:
config = configparser.RawConfigParser()
config.read('../credentials.properties')

# API creds
# here_user_id = config.get('here_api','here.user.id')
# here_client_id = config.get('here_api','here.client.id')
# here_access_key_id = config.get('here_api','here.access.key.id')
# here_access_key_secret = config.get('here_api','here.access.key.secret')
# here_token_endpoint_url = config.get('here_api','here.token.endpoint.url')
here_access_token = config.get('here_api_access','here.rest.access.token')

# DB 
db_host = config.get('db','db.host')
db_name = config.get('db','db.name')
db_user = config.get('db','db.user')
db_password = config.get('db','db.password')
db_port = config.get('db','db.port')

In [3]:
# connect with db for table writes
conn_string = "host="+ db_host +" port="+ db_port +" dbname="+ db_name +" user=" + db_user \
+" password="+ db_password
conn=psycopg2.connect(conn_string)
cur = conn.cursor()

In [4]:
# alt connection string with db
connection_str = 'postgresql+psycopg2://'+db_user+':'+db_password+'@'+db_host+':'+db_port+'/'+db_name
try:
    engine = sqlalchemy.create_engine(connection_str)
    conn1 = engine.connect()
except:
    print('Database connection error - check creds')

In [5]:
#from faker import Faker
import faker
fake = faker.Faker()
faker.Faker.seed(0)

In [6]:
aus = faker.providers.geo.Provider(fake)
print(aus.local_latlng(country_code='AU'))        

('-32.05251', '115.88782', 'Willetton', 'AU', 'Australia/Perth')


## Registration of New Users
<img src="static/userform.png" width=300 height=60 align="left"/>

Direct data capture
- name
- email (will we have validation of emails?)
- phone (will we have validation of phone numbers?)
- suburb (will we need to have an overlay? Should we ask for postcode as well?)
- password

Indirect data capture
>Do we want to do an API query at the time of user creation, so that we encode location into the database from the start?

- timestamp
- lat and lon
- whether they are an admin

e.g. there's an [automcomplete javascript api](https://developer.here.com/documentation/examples/rest/geocoding_suggestions) <br>
![autocomplete](static/autocomplete.png)

In [7]:
sql_user_table = """
DROP TABLE IF EXISTS users;

CREATE TABLE users(
    id SERIAL,
    unix_time bigint,    
    name char varying(25),
    email char varying(320),
    phone char varying(35),
    suburb text,
    password char varying(200),
    admin boolean
);
"""
cur.execute(sql_user_table)
conn.commit()

In [8]:
sql_location_table = """
DROP  TABLE IF EXISTS locations;

CREATE TABLE locations(
    id serial,
    location_type char varying(10),
    user_id int,
    request_id int,
    query text,
    provider_id char varying(100),
    provider_result json,
    latitude numeric(8,5),
    longitude numeric (8,5)
);
"""
cur.execute(sql_location_table)
conn.commit()

### testing schemas with inputs
- generate data and write to database
- inspect results

In [9]:
location_gen = faker.providers.geo.Provider(fake)
time_gen = faker.providers.date_time.Provider(fake)
name_gen = faker.providers.person.Provider(fake)
phone_gen = faker.providers.phone_number.Provider(fake)
internet_gen = faker.providers.internet.Provider(fake)
admin = False
url = "https://geocoder.ls.hereapi.com/6.2/geocode.json"
params = {
    'apiKey': here_access_token,
    'searchtext': ''
}
def user_form_input():
    result = dict()
    result['unix_time'] = time_gen.unix_time()
    result['name'] = name_gen.name()
    result['phone_number'] = phone_gen.phone_number()
    result['email'] = internet_gen.email()
    result['password'] = fake.sha256()
    result['suburb'] = location_gen.local_latlng(country_code='AU')[2]+' '+location_gen.local_latlng(country_code='AU')[3]
    result['admin'] = admin   
    params.update(searchtext= result['suburb'])
    response = requests.request('GET', url, params = params) #todo - implement an assert; possible for the request to fail
    result['provider_result'] = response.json()
    result['provider_id'] = response.json()['Response']['View'][0]['Result'][0]['Location']['LocationId']
    result['latitude'] = response.json()['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']['Latitude']
    result['longitude'] = response.json()['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']['Longitude']
    return result

In [10]:
def database_write(data):
    # users
    user_insert_sql = """
    insert into users (
        unix_time,
        name,
        email,
        phone,
        suburb,
        password,
        admin
    )
    values (
        (%s),
        (%s),
        (%s),
        (%s),
        (%s),
        (%s),
        (%s)
    )
    RETURNING id
    """
    data_tuple = (data['unix_time'],data['name'],data['email'],data['phone_number'],data['suburb'],data['password'],data['admin'])
    cur.execute(user_insert_sql,data_tuple)
    user_id = cur.fetchone()[0]
    
    # locations
    suburb_insert_sql = """
    insert into locations (
        location_type,
        query,
        user_id,
        provider_id,
        provider_result,
        latitude,
        longitude
    )
    values (
        (%s),
        (%s),
        (%s),
        (%s),
        (%s),
        (%s),
        (%s)
    )
    """
    data_tuple_2 = ('user',data['suburb'],user_id,data['provider_id'],json.dumps(data['provider_result']),data['latitude'],data['longitude'])
    cur.execute(suburb_insert_sql,data_tuple_2)    
    conn.commit()

In [11]:
# create 20 records
for _ in range(20):
    database_write(user_form_input())

In [12]:
# evaluate results
pd.read_sql('select * from users',engine)

,id,unix_time,name,email,phone,suburb,password,admin
0,1,903170602,Tammy Tucker,john89@hotmail.com,475-938-242,Baulkham Hills AU,8d35377894fb1bd8389adfdd7eeda8d9645d6a6300985e...,False
1,2,1468844309,Anita Gomez,opage@salazar-clayton.com,387-784-080,Baulkham Hills AU,2127c3a0780ab225137c370c465901499b6a6277ce70cb...,False
2,3,1218720039,Linda Thomas,lisa83@hotmail.com,115-871-484,Carnegie AU,20b1ea61ae1ddb4a99ced29558aea627a892a4c150c696...,False
3,4,1280516278,Logan Allen,wvillanueva@yahoo.com,209-471-122,Baulkham Hills AU,fad50032c7ac4a167af6ff21dfbfb6822f967a8dfacf9f...,False
4,5,505736119,Patty Lawrence,megan30@wilson.com,947-751-591,Wheelers Hill AU,f3afcae60d709d7fdc498e7cb34173ed0ef68ac9dc1c5a...,False
5,6,1498236291,Lisa Walters,fwilson@arellano.com,910-139-916,Perth AU,d0b9693060b04211b4d59b651b476b39222d971b0d755b...,False
6,7,1168736654,Craig Green,sgonzalez@massey.biz,456-208-709,Perth AU,1714e9f3159bf8965bdf51bc953715e42747f13bbad64d...,False
7,8,347793724,John Carlson,annsmith@hotmail.com,207-698-456,Perth AU,358fb875d6341ae0e9edd1e291bc03bc09e92acea4defc...,False
8,9,515676112,Timothy Bishop,johnwalsh@hutchinson-carroll.com,992-466-109,Baulkham Hills AU,3b68a29278d4e2b537af9bce8d745aa82180bafe1cb2e0...,False
9,10,1422054832,Gabriel Brown,lisagolden@lopez.com,787-890-075,Frankston South AU,5b8c984200642cddf837f2a42da8b6d3f19dab23a889d6...,False


In [13]:
pd.read_sql('select * from locations',engine)

,id,location_type,user_id,request_id,query,provider_id,provider_result,latitude,longitude
0,1,user,1,None,Baulkham Hills AU,NT_mzo-Vsv-LvLHVikKQksr2A,{'Response': {'MetaInfo': {'Timestamp': '2020-...,-33.75760,150.98960
1,2,user,2,None,Baulkham Hills AU,NT_mzo-Vsv-LvLHVikKQksr2A,{'Response': {'MetaInfo': {'Timestamp': '2020-...,-33.75760,150.98960
2,3,user,3,None,Carnegie AU,NT_BOyeiKsxCxuWd7zl.W58JB,{'Response': {'MetaInfo': {'Timestamp': '2020-...,-37.88931,145.05706
3,4,user,4,None,Baulkham Hills AU,NT_mzo-Vsv-LvLHVikKQksr2A,{'Response': {'MetaInfo': {'Timestamp': '2020-...,-33.75760,150.98960
4,5,user,5,None,Wheelers Hill AU,NT_nTi6x7lEP52HEfWLnDjX7B,{'Response': {'MetaInfo': {'Timestamp': '2020-...,-37.90559,145.18932
5,6,user,6,None,Perth AU,NT_QWrDF-VBc4mssbV9tphZbD,{'Response': {'MetaInfo': {'Timestamp': '2020-...,-31.95265,115.85743
6,7,user,7,None,Perth AU,NT_QWrDF-VBc4mssbV9tphZbD,{'Response': {'MetaInfo': {'Timestamp': '2020-...,-31.95265,115.85743
7,8,user,8,None,Perth AU,NT_QWrDF-VBc4mssbV9tphZbD,{'Response': {'MetaInfo': {'Timestamp': '2020-...,-31.95265,115.85743
8,9,user,9,None,Baulkham Hills AU,NT_mzo-Vsv-LvLHVikKQksr2A,{'Response': {'MetaInfo': {'Timestamp': '2020-...,-33.75760,150.98960
9,10,user,10,None,Frankston South AU,NT_pD8YrI--6Hk7qcGa0HMCEB,{'Response': {'MetaInfo': {'Timestamp': '2020-...,-38.17106,145.12095


### discussion
- schemas seem to work with real data
- type seems to make sense with locations
- might make sense to add a timestamp as well to location (in case there's an update clause, and we must select most recent for a given user)

## TODO
- Request data gen/schema
- Volunteer data gen/schema
- Select requests by location near volunteer (though this may be handled by the rest of the team)

# Scratch (ignore)

In [14]:
url = "https://geocoder.ls.hereapi.com/6.2/geocode.json"
params = {
    'apiKey': here_access_token,
    'searchtext': 'Gymea NSW Australia'
}
response = requests.request('GET', url, params = params)

print(response.text.encode('utf8'))

b'{"Response":{"MetaInfo":{"Timestamp":"2020-04-03T23:05:06.647+0000"},"View":[{"_type":"SearchResultsViewType","ViewId":0,"Result":[{"Relevance":1.0,"MatchLevel":"district","MatchQuality":{"Country":1.0,"State":1.0,"District":1.0},"Location":{"LocationId":"NT_2d3C5homg-ecxqBOWjzCED","LocationType":"point","DisplayPosition":{"Latitude":-34.03522,"Longitude":151.08498},"NavigationPosition":[{"Latitude":-34.03522,"Longitude":151.08498}],"MapView":{"TopLeft":{"Latitude":-34.02379,"Longitude":151.07341},"BottomRight":{"Latitude":-34.04787,"Longitude":151.09379}},"Address":{"Label":"Gymea, Sydney, NSW 2227, Australia","Country":"AUS","State":"NSW","City":"Sydney","District":"Gymea","PostalCode":"2227","AdditionalData":[{"value":"Australia","key":"CountryName"},{"value":"New South Wales","key":"StateName"}]}}}]}]}}'


In [15]:
r = response.json()
r

{'Response': {'MetaInfo': {'Timestamp': '2020-04-03T23:05:06.647+0000'},
  'View': [{'_type': 'SearchResultsViewType',
    'ViewId': 0,
    'Result': [{'Relevance': 1.0,
      'MatchLevel': 'district',
      'MatchQuality': {'Country': 1.0, 'State': 1.0, 'District': 1.0},
      'Location': {'LocationId': 'NT_2d3C5homg-ecxqBOWjzCED',
       'LocationType': 'point',
       'DisplayPosition': {'Latitude': -34.03522, 'Longitude': 151.08498},
       'NavigationPosition': [{'Latitude': -34.03522, 'Longitude': 151.08498}],
       'MapView': {'TopLeft': {'Latitude': -34.02379, 'Longitude': 151.07341},
        'BottomRight': {'Latitude': -34.04787, 'Longitude': 151.09379}},
       'Address': {'Label': 'Gymea, Sydney, NSW 2227, Australia',
        'Country': 'AUS',
        'State': 'NSW',
        'City': 'Sydney',
        'District': 'Gymea',
        'PostalCode': '2227',
        'AdditionalData': [{'value': 'Australia', 'key': 'CountryName'},
         {'value': 'New South Wales', 'key': 'StateN

In [16]:
r['Response']['View'][0]['Result']

[{'Relevance': 1.0,
  'MatchLevel': 'district',
  'MatchQuality': {'Country': 1.0, 'State': 1.0, 'District': 1.0},
  'Location': {'LocationId': 'NT_2d3C5homg-ecxqBOWjzCED',
   'LocationType': 'point',
   'DisplayPosition': {'Latitude': -34.03522, 'Longitude': 151.08498},
   'NavigationPosition': [{'Latitude': -34.03522, 'Longitude': 151.08498}],
   'MapView': {'TopLeft': {'Latitude': -34.02379, 'Longitude': 151.07341},
    'BottomRight': {'Latitude': -34.04787, 'Longitude': 151.09379}},
   'Address': {'Label': 'Gymea, Sydney, NSW 2227, Australia',
    'Country': 'AUS',
    'State': 'NSW',
    'City': 'Sydney',
    'District': 'Gymea',
    'PostalCode': '2227',
    'AdditionalData': [{'value': 'Australia', 'key': 'CountryName'},
     {'value': 'New South Wales', 'key': 'StateName'}]}}}]

In [17]:
r['Response']['View'][0]['Result'][0]['Location']['LocationId']


'NT_2d3C5homg-ecxqBOWjzCED'

In [18]:
r['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']['Latitude']

-34.03522

In [19]:
r['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']['Longitude']

151.08498

In [20]:
# OAuth to get token for 24 hours

# url = "https://account.api.here.com/oauth2/token"

# payload = 'grant_type=client_credentials'
# # headers = {
# #   'Authorization': auth_str,
# #   'Content-Type': 'application/x-www-form-urlencoded'
# # }

# headers = {
#   'Authorization': 'OAuth oauth_consumer_key="XXXXX",oauth_signature_method="HMAC-SHA256",oauth_timestamp="1585954348",oauth_nonce="frhJnKSHCbG",oauth_version="1.0",oauth_signature="XXXXXX%2FR05Q%3D"',
#   'Content-Type': 'application/x-www-form-urlencoded'
# }

# response = requests.request("POST", url, headers=headers, data = payload)

# print(response.text.encode('utf8'))

# nonce?
# oauth_signature? 
# 2 things to learn about. 